In [ ]:
import numpy as np
import pandas as pd
import sklearn
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.callbacks import EarlyStopping
import seaborn as sns



In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
data = pd.read_csv('/Users/suhaan/Downloads/dlsp-portfolio-starter-code/cover_data.csv')


labels = data.iloc[:,-1]
features = data.iloc[:, : -1]

normalizer = StandardScaler()
features = normalizer.fit_transform(features)
labels = pd.get_dummies(labels)

features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size = 0.1, random_state = 12)

In [ ]:
num = features.shape[1]
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(num, activation = 'relu'))
model.add(tf.keras.layers.Dense(64, activation = 'relu'))
model.add(tf.keras.layers.Dense(7, activation = 'softmax'))

model.compile(loss = tf.keras.losses.CategoricalCrossentropy(), 
              optimizer = tf.keras.optimizers.legacy.Adam(learning_rate = 0.001),
              metrics = [tf.keras.metrics.CategoricalCrossentropy()]
             )

early_stopping = EarlyStopping(monitor='val_loss', patience=3, min_delta = 0.004)
Fitted = model.fit(features_train, labels_train, epochs = 20, validation_data=(features_test, labels_test), 
          callbacks=[early_stopping])

plot_Fitted(Fitted, 'acc')
    plot_Fitted(Fitted, 'loss')



score = model.evaluate(features_test, labels_test, verbose=0)
print(f'Test loss: {score[0]}')
print(f'Test accuracy: {score[1]}')


label_predict = model.predict(features_test)


y_pred = np.argmax(label_predict, axis=1)
y_true = np.argmax(labels_test.values, axis=1)

class_names = ['Spruce/Fir', 'Lodgepole Pine',
               'Ponderosa Pine', 'Cottonwood/Willow',
               'Aspen', 'Douglas-fir', 'Krummholz']

print(classification_report(y_true, y_pred, target_names=class_names))


def plot_heatmap(class_names, y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    fig, ax = plt.subplots(figsize=(15, 15))
    heatmap = sns.heatmap(cm, fmt='g', cmap='Blues', annot=True, ax=ax)
    ax.set_xlabel('Predicted class')
    ax.set_ylabel('True class')
    ax.set_title('Confusion Matrix')
    ax.xaxis.set_ticklabels(class_names)
    ax.yaxis.set_ticklabels(class_names)
    plt.show()

plot_heatmap(class_names, y_true, y_pred)